In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import brier_score_loss
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn import feature_selection
from scipy.stats import uniform, randint

# Import Training Data

In [3]:
train_m = pd.read_csv('input/train_m.csv')
#train_w = pd.read_csv('input/train_w.csv')

In [4]:
train_m = train_m.drop(columns=["OppTeamID"]).set_index(["Season", "TeamID"])
#train_w = train_w.drop(columns=["OppTeamID"]).set_index(["Season", "TeamID"])

In [5]:
train_m

Seed  OppSeed  Win       EFG        TO        OR       FTR  \
Season TeamID                                                               
2003   1421      16       16    1 -0.014867  0.013773 -0.015697 -0.276214   
       1421      16        1    0 -0.009085  0.040147 -0.074055  0.009067   
       1163       5        1    0  0.028157  0.025096 -0.016591 -0.101183   
       1277       7        1    0  0.003314  0.032987 -0.078249  0.111295   
       1345       9        1    0 -0.011838  0.008439 -0.116798  0.224194   
...             ...      ...  ...       ...       ...       ...       ...   
2022   1136      16       16    0 -0.022796  0.005017  0.013298  0.094653   
       1112       1        9    1  0.070335 -0.023554 -0.023004 -0.000069   
       1371       8        9    0 -0.013110 -0.033303 -0.048326  0.030791   
       1228       4       13    1  0.003423  0.007067  0.008308  0.113432   
       1458       3       14    1 -0.077022 -0.035729 -0.007565  0.195982   

                     PF  CoachSeasons  ConfBids  KPDiff  
Season TeamID                                            
2003   1421    0.803448             6         1       5  
       1421   -1.253695             6         1     266  
       1163   -1.957143            18         4      16  
       1277   -0.324885             7         5      18  
       1345   -0.535714            18         5      31  
...                 ...           ...       ...     ...  
2022   1136    2.251515            12         1       3  
       1112    1.158088             0         3     -36  
       1371    0.387500            14         6       3  
       1228    0.511089             8         9     -54  
       1458    2.301075             6         9     -86  

[2496 rows x 11 columns]

# Objective

## Minimize brier score

In [109]:
# Baseline
y = train_m["Win"].to_numpy()
y_pred = np.array([0.5] * len(y))

In [110]:
y[0:5], y_pred[0:5]

(array([1, 0, 0, 0, 0], dtype=int64), array([0.5, 0.5, 0.5, 0.5, 0.5]))

In [111]:
losses = np.subtract(y_pred, y) ** 2
brier_score_base = losses.sum() / len(losses)
brier_score_base

0.25

# Logistic regression

In [5]:
from sklearn.linear_model import LogisticRegression

In [7]:
train_m.head()

Seed  OppSeed  Win       EFG        TO        OR       FTR  \
Season TeamID                                                               
2003   1421      16       16    1 -0.014867  0.013773 -0.015697 -0.276214   
       1421      16        1    0 -0.009085  0.040147 -0.074055  0.009067   
       1163       5        1    0  0.028157  0.025096 -0.016591 -0.101183   
       1277       7        1    0  0.003314  0.032987 -0.078249  0.111295   
       1345       9        1    0 -0.011838  0.008439 -0.116798  0.224194   

                     PF  CoachSeasons  ConfBids  KPDiff  
Season TeamID                                            
2003   1421    0.803448             6         1       5  
       1421   -1.253695             6         1     266  
       1163   -1.957143            18         4      16  
       1277   -0.324885             7         5      18  
       1345   -0.535714            18         5      31

In [6]:
X = train_m.drop(columns=["Win"])
y = train_m["Win"]

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [115]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

c:\Users\CBell\Documents\git\march_madness_2023\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [116]:
lr.score(X_test, y_test)

0.706

In [117]:
# Brier score
probs = lr.predict_proba(X_test)
probs = probs[:, 1]

In [118]:
loss = brier_score_loss(y_test, probs)
loss

0.18812113882745946

In [119]:
# Skill score
1 - loss / brier_score_base

0.24751544469016218

In [120]:
# Test model cross validation
lr = LogisticRegression()
scores = cross_val_score(lr, X, y, cv = 5, scoring='neg_brier_score')
scores.mean() * -1

c:\Users\CBell\Documents\git\march_madness_2023\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\CBell\Documents\git\march_madness_2023\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.1893208784329982

## Improve model

### Make preprocessing pipeline

In [121]:
lr_pipeline = make_pipeline(preprocessing.StandardScaler(), LogisticRegression())

In [122]:
scores = cross_val_score(lr_pipeline, X, y, cv=5, scoring='neg_brier_score')
scores.mean() * -1

0.1896970772180927

### Feature selection

In [123]:
lr_pipeline.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [124]:
lr_coefs = lr_pipeline.named_steps['logisticregression'].coef_[0]

In [125]:
X_train.columns

Index(['Seed', 'OppSeed', 'EFG', 'TO', 'OR', 'FTR', 'PF', 'CoachSeasons',
       'ConfBids', 'KPDiff'],
      dtype='object')

In [126]:
pd.DataFrame({'coef': lr_coefs}, index=X_train.columns)

,coef
Seed,-0.379607
OppSeed,0.343186
EFG,0.021220
TO,-0.122069
OR,0.155090
FTR,-0.051265
PF,-0.059630
CoachSeasons,0.056534
ConfBids,-0.092956
KPDiff,-0.789771


In [10]:
X2 = X.drop(columns=['CoachSeasons', 'ConfBids'])

In [7]:
X3 = X.filter(["Seed", "OppSeed", "TO", "OR", "KPDiff"])

In [8]:
lr_pipeline = make_pipeline(preprocessing.StandardScaler(), LogisticRegression())

In [11]:
scores2 = cross_val_score(lr_pipeline, X2, y, cv=5, scoring='neg_brier_score')
scores2.mean() * -1

0.18910059774568638

In [12]:
scores3 = cross_val_score(lr_pipeline, X3, y, cv=5, scoring='neg_brier_score')
scores3.mean() * -1

0.1895485643734318

In [13]:
# Accuracy
cross_val_score(lr_pipeline, X2, y, cv=5).mean()

0.7111430861723447

In [14]:
cross_val_score(lr_pipeline, X3, y, cv=5).mean()

0.708743887775551

Use reduced feature set (X2)

## Tuning

In [82]:
LogisticRegression().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [15]:
params = {
    'logisticregression__C': [0.01, 0.1, 1, 10, 100, 1000],
    'logisticregression__penalty': [None, 'l2']
}

In [16]:
# Grid search
lr_pipeline = make_pipeline(preprocessing.StandardScaler(), LogisticRegression())
lr_cv = GridSearchCV(lr_pipeline, params, scoring='neg_brier_score', n_jobs = -1)
lr_cv.fit(X3, y)

GridSearchCV(estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('logisticregression',
                                        LogisticRegression())]),
             n_jobs=-1,
             param_grid={'logisticregression__C': [0.01, 0.1, 1, 10, 100, 1000],
                         'logisticregression__penalty': [None, 'l2']},
             scoring='neg_brier_score')

In [17]:
lr_cv.best_score_

-0.1895485643734318

In [18]:
lr_cv.best_params_

{'logisticregression__C': 1, 'logisticregression__penalty': 'l2'}

## Final model

In [19]:
# Using X3
X_final = X[X3.columns]
X_final

Seed  OppSeed        TO        OR  KPDiff
Season TeamID                                           
2003   1421      16       16  0.013773 -0.015697       5
       1421      16        1  0.040147 -0.074055     266
       1163       5        1  0.025096 -0.016591      16
       1277       7        1  0.032987 -0.078249      18
       1345       9        1  0.008439 -0.116798      31
...             ...      ...       ...       ...     ...
2022   1136      16       16  0.005017  0.013298       3
       1112       1        9 -0.023554 -0.023004     -36
       1371       8        9 -0.033303 -0.048326       3
       1228       4       13  0.007067  0.008308     -54
       1458       3       14 -0.035729 -0.007565     -86

[2496 rows x 5 columns]

In [18]:
X_final.to_csv("input/X_m.csv", index=False)

In [20]:
# Final Pipeline
scaler = preprocessing.StandardScaler()
lr = LogisticRegression(C=1, penalty='l2')
lr_pipeline = make_pipeline(scaler, lr)
lr_pipeline.fit(X_final, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(C=1))])

## Save model

In [25]:
import pickle

In [22]:
filename = open('models/lr_m.sav', mode='wb')
pickle.dump(lr_pipeline, filename)
filename.close()

# XGBoost

In [8]:
import xgboost as xgb

In [17]:
# Paramaters for tuning
xgb_params = {
    'learning_rate': uniform(0.03, 0.3),
    'max_depth': randint(2, 5),
    'gamma': uniform(0, 0.5),
    'n_estimators': randint(50, 150),
    'subsample': uniform(0.6, 0.4)
}

In [18]:
xgb_model = xgb.XGBClassifier()

In [19]:
# Cross validation
xgb_search = RandomizedSearchCV(xgb_model, 
                                param_distributions=xgb_params,
                                n_iter = 200,
                                cv=4,
                                verbose=1,
                                scoring='neg_brier_score')

In [20]:
xgb_search.fit(X3, y)

Fitting 4 folds for each of 200 candidates, totalling 800 fits


RandomizedSearchCV(cv=4,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000235C465FDF0>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000235BCE2AC40>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000235C3656190>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000235C366D730>},
                   scoring='neg_brier_score', verbose=1)

In [21]:
xgb_search.best_score_

-0.18841181937373042

In [22]:
xgb_search.best_params_

{'gamma': 0.48571359662583274,
 'learning_rate': 0.0501168129627176,
 'max_depth': 2,
 'n_estimators': 91,
 'subsample': 0.7204228644291942}

In [23]:
xgb_model_final = xgb_search.best_estimator_

In [26]:
filename = open('models/xgb_m.sav', mode='wb')
pickle.dump(xgb_model_final, filename)
filename.close()